In [ ]:
from datetime import date, datetime, timedelta
import girder_client as gc
import json
import math
import os
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.ticker import PercentFormatter
from urllib.parse import quote
import random
import re
import string
import tzlocal
from urllib.request import urlopen

with open("config.json", "r") as fp:
    config=json.load(fp)

In [ ]:
version_dates = [
    date(2018, 12, 21),
    date(2019, 2, 20)
]
version_assignments = [
    7,
    2
]
days_between_versions = [
    (version_dates[v+1] - version_dates[v]).days for v in range(len(version_dates)-1)
]

In [ ]:
def plot_responses(responses):
    """
    Parameters
    ----------
    responses: dictionary
       responses[user]["created"]: date
           date user account created
       responses[user][key]: string
           assessment name
       responses[user][key]: value: list
           list of ints, number of responses per day
    
    Return
    ------
    displays and saves PyPlot figure or returns None if no responses
    """
    oldest = max([
        version_dates[0],
        min([pd.to_datetime(
            girder_connection.getUser(user)["created"]
        ).date() for user in responses.keys()])
    ])
    date_lines = [(d - oldest).days for d in version_dates]
    
    for user in responses:
        responses[user]['percent'] = []
        for i, d in enumerate(responses[user]['counts']):
            for j, v in enumerate(version_assignments):
                if j < (len(days_between_versions)):
                    put_in_list(responses[user]['percent'], i, d/(
                        v if i < days_between_versions[
                            j
                        ] else version_assignments[j+1]
                    ))
                    
    for user in responses:
        for i, v in enumerate(responses[user]['percent']):
            responses[user]['percent'][i] = min(1, v)
    
    fig, axes = plt.subplots(1, 1)
    fig.suptitle(
        "Completed / Assigned"
    )
    fig.set_figwidth(20)
    xticks = range(
        0,
        (
            date.today() - oldest
        ).days
    )
    xlabels = [
        (
            oldest + timedelta(x)
        ).strftime("%m-%d") for x in xticks
    ]
    axes.set_xticks(xticks)
    axes.set_xticklabels(xlabels)
    axes.tick_params(
        axis='x',
        rotation=90
    )
    for i, user in enumerate(responses):
        axes.plot(responses[user]['percent'])
    for date_line in [date_line for date_line in date_lines if date_line > 0]:
        axes.axvline(date_line)
    axes.yaxis.set_major_formatter(PercentFormatter(1))
    imgPath = "reports/user_response_charts/"
    if not os.path.exists(imgPath):
        os.makedirs(imgPath)
    plt.savefig(
        "{}/{}-{}.png".format(
            imgPath,
            datetime.now().strftime("%Y-%m-%d-%H-%M"),
            'percents'
        ),
        dpi=150,
        bbox_inches='tight'
    )
    plt.show()
    plt.close()

In [ ]:
girder_connection = gc.GirderClient(
    apiUrl="{}/api/v1/".format(
        config["girder-production"]["host"]
    )
)
girder_connection.authenticate(
    username=config["girder-production"]["user"],
    password=config["girder-production"]["password"]
)

In [ ]:
ema_parent_users = girder_connection.getFolder("5bd87caa336da80de9145af2")["meta"]["members"]["users"]

In [ ]:
allResponses = {}
for user in ema_parent_users:
    allResponses[user] = girder_connection.get("response?userId={}&appletId=5bd87caa336da80de9145af2".format(user))

In [ ]:
allResponses

In [ ]:
counts = {}
for user in allResponses:
    girder_user = girder_connection.getUser(user)
    counts[user] = {
        'created': pd.to_datetime(
            girder_user['created']
        ).date(),
        'name': girder_user['firstName']
    }
    for response in allResponses[user]:
        for innerResponse in allResponses[user][response]:
            if innerResponse['meta']['activity']['name'] not in counts[user]:
                counts[user][innerResponse['meta']['activity']['name']] = []
            counts[user][innerResponse['meta']['activity']['name']].append(
                pd.to_datetime(
                    innerResponse['meta']['responseTime'],
                    unit="ms"
                ).date()
            )

In [ ]:
def put_in_list(l, index, value):
    if not(type(l)) == list:
        l = []
    if len(l) > index:
        l[index] = l[index] + value if l[index] is not None else value 
    elif len(l) == index:
        l.append(value)
    else:
        l = l + (
            [0] * (index - len(l))
        )
        l.append(value)
    return(l)

In [ ]:
responses = {}
total_days = (date.today() - version_dates[0]).days
for user in allResponses:
    girder_user = girder_connection.getUser(user)
    responses[user] = {
        'created': pd.to_datetime(
            girder_user['created']
        ).date(),
        'name': girder_user['firstName']
    }
    for applet in allResponses[user]:
        for response in allResponses[user][applet]:
            day = (pd.to_datetime(
                response['meta']['responseTime'],
                unit='ms'
            ).date() - version_dates[0]).days
            if day >= 0:
                responses[user][
                    response['meta']['activity']['name']
                ] = put_in_list(
                    responses[user][
                        response['meta']['activity']['name']
                    ] if response['meta']['activity']['name'] in responses[
                        user
                    ] else [],
                    day,
                    value=1
                )
    for activity in [activity for activity in responses[user] if activity not in [
        "created", "name"
    ]]:
        if len(responses[user][activity]) < total_days:
            responses[user][activity] += ([0] * (total_days - len(responses[user][activity])))

In [ ]:
df = pd.DataFrame(responses).T
all_activities = [
    person for person in df[[col for col in df.columns if col not in ['created', 'name']]]
]
for col in all_activities:
    df.loc[df[col].isnull(),[col]] = df.loc[df[col].isnull(),col].apply(lambda x: [0] * total_days)

In [ ]:
counts = [list(map(sum, zip(*person))) for person in df[
    [col for col in df.columns if col not in ['created', 'name']]
].values.tolist()]

user_counts = {}

for i in range(df.shape[0]):
    user_counts[df.index[i]] = {
        'name': df.loc[df.index[i]]['name'],
        'created': df.loc[df.index[i]].created,
        'counts': counts[i]
    }
plot_responses(user_counts)